In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define paths
dataset_dir = 'Emotions'

# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

# Create train and validation generators
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))  # 6 output classes for the emotions

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=100  # You can adjust the number of epochs
)

# Save the model
model.save('emotion_recognition_model.h5')


Found 920 images belonging to 6 classes.
Found 226 images belonging to 6 classes.


C:\Users\aryah\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 306ms/step - accuracy: 0.1942 - loss: 1.7938 - val_accuracy: 0.3036 - val_loss: 1.7828
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0938 - loss: 1.8082 - val_accuracy: 0.0000e+00 - val_loss: 1.8340
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 295ms/step - accuracy: 0.2075 - loss: 1.7842 - val_accuracy: 0.2054 - val_loss: 1.7752
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - accuracy: 0.2500 - loss: 1.7851 - val_accuracy: 0.0000e+00 - val_loss: 1.9223
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 260ms/step - accuracy: 0.2008 - loss: 1.7843 - val_accuracy: 0.1964 - val_loss: 1.7729
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.1562 - loss: 1.7589 - val_accuracy: 0.0000e+00 - val_loss: 1.6214
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 252ms/step - accuracy: 0.1734 - loss: 1.7747 - val_accuracy: 0.2723 - val_loss: 1.7682
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2500 - loss: 1.7740

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('emotion_recognition_model.h5')

# Define the class labels (these should match the folder names in the dataset)
class_labels = ['happy', 'anger', 'pain', 'disgust', 'fear', 'sad']

def predict_emotion(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    # Predict the emotion
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = class_labels[predicted_class_index]

    return predicted_class

# Example usage:
img_path = r'Emotions\happyface.jpg'  # Raw string to avoid escape issues
predicted_emotion = predict_emotion(img_path)
print(f'The predicted emotion is: {predicted_emotion}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
The predicted emotion is: happy
